# Description

This notebook will prepare data by convert Health Heritage Prize Data Release 3 in Claim level with categories type to Member level with one-hot type and summing type (for columns with large number value).
Each column in Claims will convert and save to csv file for later use, due to long time processing.

Files input:

1. Members.csv
2. Claims.csv
3. DrugCount.csv
4. LabCount.csv
5. DaysInHospital_Y2.csv
6. DaysInHospital_Y3.csv



Files output:

- MemberInfo_df.csv
- DaysInHos_Y2.csv
- DaysInHos_Y3.csv
- Claims_count_Y1.csv
- Claims_count_Y2.csv
- Charlson_Y1.csv
- Charlson_Y2.csv
- DrugCount_Y1.csv
- DrugCount_Y2.csv
- DSFS_Y1.csv
- DSFS_Y2.csv
- LabCount_Y1.csv
- LabCount_Y2.csv
- LOS_Y1.csv
- LOS_Y2.csv
- PayDelay_Y1.csv
- PayDelay_Y2.csv
- PCPs_Y1.csv
- PCPs_Y2.csv
- PlaceSvc_Y1.csv
- PlaceSvc_Y2.csv
- PrimCondition_Y1.csv
- PrimCondition_Y2.csv
- Procedures_Y1.csv
- Procedures_Y2.csv
- Providers_Y1.csv
- Providers_Y2.csv
- Specialty_Y1.csv
- Specialty_Y2.csv
- Vendors_Y1.csv
- Vendors_Y2.csv

# Import Library

In [2]:
import pandas as pd
import numpy as np

# 1. Member_Info Process
- Sex: One-Hot with Male, Female, LGBT
- AgeAtFirstClaim: replace categories string by integer mean average, exp. '10-19' to 15

In [12]:
MemInfo_df = pd.read_csv('data/hhp_release3/Members.csv')

# One-Hot to Male, Female and LGBT
MemInfo_df['Male'] = 0
MemInfo_df['Female'] = 0
MemInfo_df['LGBT'] = 0
MemInfo_df.loc[MemInfo_df.Sex == 'M', 'Male'] = 1
MemInfo_df.loc[MemInfo_df.Sex == 'F', 'Female'] = 1
MemInfo_df.loc[MemInfo_df.Sex.isna(), 'LGBT'] = 1
MemInfo_df.drop(columns=['Sex'], inplace=True)

# Replace Age with mean
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '0-9', 'AgeAtFirstClaim'] = 5
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '10-19', 'AgeAtFirstClaim'] = 15
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '20-29', 'AgeAtFirstClaim'] = 25
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '30-39', 'AgeAtFirstClaim'] = 35
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '40-49', 'AgeAtFirstClaim'] = 45
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '50-59', 'AgeAtFirstClaim'] = 55
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '60-69', 'AgeAtFirstClaim'] = 65
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '70-79', 'AgeAtFirstClaim'] = 75
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim == '80+', 'AgeAtFirstClaim'] = 80
MemInfo_df.loc[MemInfo_df.AgeAtFirstClaim.isna(), 'AgeAtFirstClaim'] = 45 # Average

# Save to CSV file
MemInfo_df.to_csv(path_or_buf='processed_data/MemberInfo_df.csv', index=False)

 # 2. Claims processing
 
Claims table is claims level information with features:
 - MemberID: Patient ID
 - ProviderID: Claims provider ID
 - Vendor: Vendor ID
 - PCP: PCP ID
 - Year: Y1, Y2 or Y3
 - Specialty: Categorical data of Specialty
 - PlaceSvc: Categorical data of service place
 - PayDelay: Number in range 0 - 161 with 95% percentile top-coded as string '162+'
 - LengthOfSay: Categorical data of day length up to 6 days and weeks length range up to 8 weeks. If above that length, Supression is applied and value at SupLOS column is 1, else 0
 - DSFS: Categorical data of number of month, range from 1 to 12.
 - PrimaryConditionGroup: Categorical data of diagnostic 
 - CharlsonIndex: Categorical data of CharlsonIndex
 - ProcedureGroup: Categorical data of procedures
 - SupLOS: 0 and 1 for none supresstion and suppressed in Length of Stay values, respectively.

This processing step will only use Y1 data for training and Y2 for testing. Each will convert features from claims level to member level following rules:
 - ID data like Provider, Vendor and PCP will sum the unique values for each MemberID.
 - Numeric data like PayDelay will convert to integer and sum for each MemberID.
 - Other Categorical data will One-Hot with value_counts based on MemberID.
 
Each features will be converted and save to seperated csv file.

## 2.0. Import and Split (Y1)Train / (Y2)Test

In [9]:
claims_Y12_df = pd.read_csv('data/hhp_release3/Claims.csv').query("Year == 'Y1' or Year == 'Y2'")
claims_Y1 = claims_Y12_df.query("Year == 'Y1'")
claims_Y2 = claims_Y12_df.query("Year == 'Y2'")

## 2.1. Claims Count

In [4]:
def Claims_count(df, length=None):
    """
    df: DataFrame (Y1_df or Y2_df)
    length: int, slice [0: length] to tranfer, None for full
    return pd.DataFrame with columns=['MemberID', 'ClaimsCount']
    """
    length = df.MemberID.unique().shape[0] + 1 if length is None else length
    member_list_count = list()
    for mem_id, count in df.MemberID.value_counts().items():
        member_list_count.append([mem_id, count])

    return pd.DataFrame(np.array(member_list_count), columns=['MemberID', 'ClaimsCount'])

In [5]:
Claims_countY1_df = Claims_count(claims_Y1)
Claims_countY2_df = Claims_count(claims_Y2)

# Save to CSV
Claims_countY1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Claims_count_Y1.csv', index=False)
Claims_countY2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Claims_count_Y2.csv', index=False)

## 2.2.  Count number of Unique ID

- Provider
- Vendor
- PCP

In [79]:
def unique_counter(claims_df, claims_count_df, feature_name, length=None):
    """
    claims_df: DataFrame (claims_Y1_df or claims_Y2_df)
    length: int, slice [0: length] to tranfer, None for full
    return pd.DataFrame with columns=['MemberID', 'Unique Providers']
    """
    length = claims_count_df.shape[0] if length is None else length
    member_list_count = list()
    for mem_id in claims_count_df.MemberID:
        feature_count = claims_df[claims_df.MemberID == mem_id][feature_name].unique().shape[0]
        member_list_count.append([mem_id, feature_count])
        if len(member_list_count) % 1000 == 0:
            print(len(member_list_count), '/', length)
        if len(member_list_count) == length:
            break
            
    return pd.DataFrame(np.array(member_list_count), columns=['MemberID', "Unique " + feature_name])

### 2.2.1. Providers

In [80]:
ProvidersY1_df = unique_counter(claims_Y1, Claims_countY1_df, 'ProviderID')
ProvidersY2_df = unique_counter(claims_Y2, Claims_countY2_df, 'ProviderID')

# Save to CSV
ProvidersY1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Providers_Y1.csv', index=False)
ProvidersY2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Providers_Y2.csv', index=False)

1000 / 76038
2000 / 76038
3000 / 76038
4000 / 76038
5000 / 76038
6000 / 76038
7000 / 76038
8000 / 76038
9000 / 76038
10000 / 76038
11000 / 76038
12000 / 76038
13000 / 76038
14000 / 76038
15000 / 76038
16000 / 76038
17000 / 76038
18000 / 76038
19000 / 76038
20000 / 76038
21000 / 76038
22000 / 76038
23000 / 76038
24000 / 76038
25000 / 76038
26000 / 76038
27000 / 76038
28000 / 76038
29000 / 76038
30000 / 76038
31000 / 76038
32000 / 76038
33000 / 76038
34000 / 76038
35000 / 76038
36000 / 76038
37000 / 76038
38000 / 76038
39000 / 76038
40000 / 76038
41000 / 76038
42000 / 76038
43000 / 76038
44000 / 76038
45000 / 76038
46000 / 76038
47000 / 76038
48000 / 76038
49000 / 76038
50000 / 76038
51000 / 76038
52000 / 76038
53000 / 76038
54000 / 76038
55000 / 76038
56000 / 76038
57000 / 76038
58000 / 76038
59000 / 76038
60000 / 76038
61000 / 76038
62000 / 76038
63000 / 76038
64000 / 76038
65000 / 76038
66000 / 76038
67000 / 76038
68000 / 76038
69000 / 76038
70000 / 76038
71000 / 76038
72000 / 76038
7

### 2.2.2. Vendors

In [ ]:
VendorsY1_df = unique_counter(claims_Y1, Claims_countY1_df, 'Vendor')
VendorsY2_df = unique_counter(claims_Y2, Claims_countY2_df, 'Vendor')

# Save to CSV
VendorsY1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Vendors_Y1.csv', index=False)
VendorsY2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Vendors_Y2.csv', index=False)

### 2.2.3. PCPs

In [ ]:
PCPsY1_df = unique_counter(claims_Y1, Claims_countY1_df, 'PCP')
PCPsY2_df = unique_counter(claims_Y2, Claims_countY2_df, 'PCP')

# Save to CSV
PCPsY1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PCPs_Y1.csv', index=False)
PCPsY2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PCPs_Y2.csv', index=False)

## 2.3. One-Hot with value_counts for categories feature

- Specialty
- PlaceSvc
- PrimaryConditionGroup
- CharlsonIndex
- ProcedureGroup
- DSFS

In [24]:
def one_hot_value_counts(claim_count_df, claims_df, claimY_df, col_name):
    # add base-0 column
    feature_df = pd.DataFrame(claim_count_df.MemberID)
    for value, count in claims_df[col_name].value_counts().items():
        feature_df[value] = 0
    print('Added {} base column'.format(feature_df.shape[1] - 1))
    
    i = 0
    n = len(feature_df)
    # count and add count value to column name keys
    for mem_id in claim_count_df.MemberID:
        for name, count in claimY_df[claimY_df.MemberID == mem_id][col_name].value_counts().items():
            feature_df.loc[feature_df.MemberID == mem_id, str(name)] = count
        # print out
        i += 1
        if i % 1000 == 0:
            print(i, '/', n)
    feature_df.rename(columns={"Other": "Other" + col_name}, inplace=True)
    print('DONE')
    
    return feature_df

### 2.3.1. Specialty

In [20]:
SpecialtyY1_df = one_hot_value_counts(Claims_countY1_df, claims_Y12_df, claims_Y1, 'Specialty')
SpecialtyY2_df = one_hot_value_counts(Claims_countY2_df, claims_Y12_df, claims_Y2, 'Specialty')

# Save to CSV
SpecialtyY1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Specialty_Y1.csv', index=False)
SpecialtyY2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Specialty_Y2.csv', index=False)

Added 12 base column
1000 / 71435
2000 / 71435
3000 / 71435
4000 / 71435
5000 / 71435
6000 / 71435
7000 / 71435
8000 / 71435
9000 / 71435
10000 / 71435
11000 / 71435
12000 / 71435
13000 / 71435
14000 / 71435
15000 / 71435
16000 / 71435
17000 / 71435
18000 / 71435
19000 / 71435
20000 / 71435
21000 / 71435
22000 / 71435
23000 / 71435
24000 / 71435
25000 / 71435
26000 / 71435
27000 / 71435
28000 / 71435
29000 / 71435
30000 / 71435
31000 / 71435
32000 / 71435
33000 / 71435
34000 / 71435
35000 / 71435
36000 / 71435
37000 / 71435
38000 / 71435
39000 / 71435
40000 / 71435
41000 / 71435
42000 / 71435
43000 / 71435
44000 / 71435
45000 / 71435
46000 / 71435
47000 / 71435
48000 / 71435
49000 / 71435
50000 / 71435
51000 / 71435
52000 / 71435
53000 / 71435
54000 / 71435
55000 / 71435
56000 / 71435
57000 / 71435
58000 / 71435
59000 / 71435
60000 / 71435
61000 / 71435
62000 / 71435
63000 / 71435
64000 / 71435
65000 / 71435
66000 / 71435
67000 / 71435
68000 / 71435
69000 / 71435
70000 / 71435
71000 / 

### 2.3.2. PlaceSvc

In [21]:
PlaceSvc_Y1_df = one_hot_value_counts(Claims_countY1_df, claims_Y12_df, claims_Y1, 'PlaceSvc')
PlaceSvc_Y2_df = one_hot_value_counts(Claims_countY2_df, claims_Y12_df, claims_Y2, 'PlaceSvc')

# Save to CSV
PlaceSvc_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PlaceSvc_Y1.csv', index=False)
PlaceSvc_Y2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PlaceSvc_Y2.csv', index=False)

Added 8 base column
1000 / 71435
2000 / 71435
3000 / 71435
4000 / 71435
5000 / 71435
6000 / 71435
7000 / 71435
8000 / 71435
9000 / 71435
10000 / 71435
11000 / 71435
12000 / 71435
13000 / 71435
14000 / 71435
15000 / 71435
16000 / 71435
17000 / 71435
18000 / 71435
19000 / 71435
20000 / 71435
21000 / 71435
22000 / 71435
23000 / 71435
24000 / 71435
25000 / 71435
26000 / 71435
27000 / 71435
28000 / 71435
29000 / 71435
30000 / 71435
31000 / 71435
32000 / 71435
33000 / 71435
34000 / 71435
35000 / 71435
36000 / 71435
37000 / 71435
38000 / 71435
39000 / 71435
40000 / 71435
41000 / 71435
42000 / 71435
43000 / 71435
44000 / 71435
45000 / 71435
46000 / 71435
47000 / 71435
48000 / 71435
49000 / 71435
50000 / 71435
51000 / 71435
52000 / 71435
53000 / 71435
54000 / 71435
55000 / 71435
56000 / 71435
57000 / 71435
58000 / 71435
59000 / 71435
60000 / 71435
61000 / 71435
62000 / 71435
63000 / 71435
64000 / 71435
65000 / 71435
66000 / 71435
67000 / 71435
68000 / 71435
69000 / 71435
70000 / 71435
71000 / 7

### 2.3.3. PrimaryConditionGroup

In [22]:
PrimCondition_Y1_df = one_hot_value_counts(Claims_countY1_df, claims_Y12_df, claims_Y1, 'PrimaryConditionGroup')
PrimCondition_Y2_df = one_hot_value_counts(Claims_countY2_df, claims_Y12_df, claims_Y2, 'PrimaryConditionGroup')

# Save to CSV
PrimCondition_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PrimCondition_Y1.csv', index=False)
PrimCondition_Y2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PrimCondition_Y2.csv', index=False)

Added 45 base column
1000 / 71435
2000 / 71435
3000 / 71435
4000 / 71435
5000 / 71435
6000 / 71435
7000 / 71435
8000 / 71435
9000 / 71435
10000 / 71435
11000 / 71435
12000 / 71435
13000 / 71435
14000 / 71435
15000 / 71435
16000 / 71435
17000 / 71435
18000 / 71435
19000 / 71435
20000 / 71435
21000 / 71435
22000 / 71435
23000 / 71435
24000 / 71435
25000 / 71435
26000 / 71435
27000 / 71435
28000 / 71435
29000 / 71435
30000 / 71435
31000 / 71435
32000 / 71435
33000 / 71435
34000 / 71435
35000 / 71435
36000 / 71435
37000 / 71435
38000 / 71435
39000 / 71435
40000 / 71435
41000 / 71435
42000 / 71435
43000 / 71435
44000 / 71435
45000 / 71435
46000 / 71435
47000 / 71435
48000 / 71435
49000 / 71435
50000 / 71435
51000 / 71435
52000 / 71435
53000 / 71435
54000 / 71435
55000 / 71435
56000 / 71435
57000 / 71435
58000 / 71435
59000 / 71435
60000 / 71435
61000 / 71435
62000 / 71435
63000 / 71435
64000 / 71435
65000 / 71435
66000 / 71435
67000 / 71435
68000 / 71435
69000 / 71435
70000 / 71435
71000 / 

### 2.3.4. CharlsonIndex

In [105]:
Charlson_Y1_df = one_hot_value_counts(Claims_countY1_df, claims_Y12_df, claims_Y1, 'CharlsonIndex')
Charlson_Y2_df = one_hot_value_counts(Claims_countY2_df, claims_Y12_df, claims_Y2, 'CharlsonIndex')

# Save to CSV
Charlson_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Charlson_Y1.csv', index=False)
Charlson_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Charlson_Y2.csv', index=False)

Added 4 base column
10000 / 76038
20000 / 76038
30000 / 76038
40000 / 76038
50000 / 76038
60000 / 76038
70000 / 76038
DONE
Added 4 base column
10000 / 76038
20000 / 76038
30000 / 76038
40000 / 76038
50000 / 76038
60000 / 76038
70000 / 76038
DONE


### 2.3.5. ProcedureGroup

In [17]:
Procedures_Y1_df = one_hot_value_counts(Claims_countY1_df, claims_Y12_df, claims_Y1, 'ProcedureGroup')
Procedures_Y2_df = one_hot_value_counts(Claims_countY2_df, claims_Y12_df, claims_Y2, 'ProcedureGroup')

# Save to CSV
Procedures_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Procedures_Y1.csv', index=False)
Procedures_Y2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/Procedures_Y2.csv', index=False)

Added 17 base column
1000 / 76038


KeyboardInterrupt: 

### 2.3.6. DSFS

In [25]:
DSFS_Y1_df = one_hot_value_counts(Claims_countY1_df, claims_Y12_df, claims_Y1, 'DSFS')
DSFS_Y2_df = one_hot_value_counts(Claims_countY2_df, claims_Y12_df, claims_Y2, 'DSFS')

# Save to CSV
DSFS_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/DSFS_Y1.csv', index=False)
DSFS_Y2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/DSFS_Y2.csv', index=False)

Added 12 base column
1000 / 76038
2000 / 76038
3000 / 76038
4000 / 76038
5000 / 76038
6000 / 76038
7000 / 76038
8000 / 76038
9000 / 76038
10000 / 76038
11000 / 76038
12000 / 76038
13000 / 76038
14000 / 76038
15000 / 76038
16000 / 76038
17000 / 76038
18000 / 76038
19000 / 76038
20000 / 76038
21000 / 76038
22000 / 76038
23000 / 76038
24000 / 76038
25000 / 76038
26000 / 76038
27000 / 76038
28000 / 76038
29000 / 76038
30000 / 76038
31000 / 76038
32000 / 76038
33000 / 76038
34000 / 76038
35000 / 76038
36000 / 76038
37000 / 76038
38000 / 76038
39000 / 76038
40000 / 76038
41000 / 76038
42000 / 76038
43000 / 76038
44000 / 76038
45000 / 76038
46000 / 76038
47000 / 76038
48000 / 76038
49000 / 76038
50000 / 76038
51000 / 76038
52000 / 76038
53000 / 76038
54000 / 76038
55000 / 76038
56000 / 76038
57000 / 76038
58000 / 76038
59000 / 76038
60000 / 76038
61000 / 76038
62000 / 76038
63000 / 76038
64000 / 76038
65000 / 76038
66000 / 76038
67000 / 76038
68000 / 76038
69000 / 76038
70000 / 76038
71000 / 

## 2.4. Other

### 2.4.1. LengthOfStay with SupLOS

In [ ]:
def LOS_with_SupLOS_count(claim_count_df, claims_df, claimY_df):
    # add base-0 column
    feature_df = pd.DataFrame(claim_count_df.MemberID)
    for value in claims_df.LengthOfStay.unique():
        feature_df['LOS_' + str(value)] = 0
    
    print('Added {} base column'.format(feature_df.shape[1] - 1))
    i = 0
    n = len(feature_df)
    # count and add count value to column name keys
    for mem_id in claim_count_df.MemberID:
        for name, count in claimY_df[claimY_df.MemberID == mem_id].LengthOfStay.value_counts().items():
            if str(name) is not 'nan':
                feature_df.loc[feature_df.MemberID == mem_id, 'LOS_' + str(name)] = count
        
        feature_df.loc[feature_df.MemberID == mem_id, 'LOS_nan'] = \
            len(claimY_df[claimY_df.MemberID == mem_id].query("SupLOS == 0 and LengthOfStay.isna() == True"))
        mem_df.loc[mem_df.MemberID == mem_id, 'SupLOS_nan'] = \
            len(claimY_df[claimY_df.MemberID == mem_id].query("SupLOS == 1"))
            
        # print out
        i += 1
        if i % 1000 == 0:
            print(i, '/', n)

    print('DONE')
    return feature_df

In [23]:
LOS_Y1_df = LOS_with_SupLOS_count(Claims_countY1_df, claims_Y12_df, claims_Y1)
LOS_Y2_df = LOS_with_SupLOS_count(Claims_countY2_df, claims_Y12_df, claims_Y2)

# Save to CSV
LOS_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/LOS_Y1.csv', index=False)
LOS_Y2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/LOS_Y2.csv', index=False)

Added 9 base column
1000 / 76038
2000 / 76038
3000 / 76038
4000 / 76038


KeyboardInterrupt: 

### 2.4.2. PayDelay

In [31]:
# Replace PayDelay 162+ with 162
claims_Y1.PayDelay.replace({'162+': '162'}, inplace=True)
claims_Y2.PayDelay.replace({'162+': '162'}, inplace=True)

# Change to int with downcast
claims_Y1['PayDelay'] = pd.to_numeric(claims_Y1.PayDelay, downcast='integer')
claims_Y2['PayDelay'] = pd.to_numeric(claims_Y2.PayDelay, downcast='integer')

0           28
6          162
7           29
11          51
12          22
14         143
15         162
16          62
18          46
21          24
22         162
26         162
31          17
32          28
35          35
51          49
54          93
55          23
56          16
57          29
58          74
61         162
65         135
66          23
69         112
70          65
73          29
74          36
76         148
78          63
          ... 
2668892     80
2668899     55
2668906     21
2668907     51
2668910     30
2668911     28
2668914     35
2668917    129
2668918    162
2668919     62
2668920     21
2668924     62
2668927     29
2668928     37
2668930     36
2668932     44
2668937     37
2668939     69
2668944     31
2668948     12
2668949     17
2668950     74
2668951     59
2668958     29
2668965     37
2668972     44
2668975     17
2668977     35
2668978     38
2668989     15
Name: PayDelay, Length: 865689, dtype: int64

In [ ]:
def PayDelay_summing(claim_count_df, claimY_df):
    print('PayDelay summing')
    feature_df = pd.DataFrame(claim_count_df.MemberID)
    feature_df['PayDelay'] = 0
    
    i = 0
    n = len(feature_df)
    # count and sum
    for mem_id in feature_df.MemberID:
        feature_df.loc[feature_df.MemberID == mem_id, 'PayDelay'] = \
            claimY_df[claimY_df.MemberID == mem_id].PayDelay.sum()
        
        # print out
        i += 1
        if i % 1000 == 0:
            print(i, '/', n)
    
    print('DONE')
    return feature_df

In [ ]:
PayDelay_Y1_df = PayDelay_summing(Claims_countY1_df, claims_Y1)
PayDelay_Y2_df = PayDelay_summing(Claims_countY2_df, claims_Y2)

# Save to CSV
PayDelay_Y1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PayDelay_Y1.csv', index=False)
PayDelay_Y2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/PayDelay_Y2.csv', index=False)

# 3 & 4. Drug and Lab Count

- DrugCount is count of unique prescription drugs filled and top-coded at 7
- LabCount is count of unique laboratory and pathology tests and top-coded at 10

Both is numeric data and will be converted to integer and sum for each MemberID for each Year.

In [60]:
def Count_summing(df, claims_df, feature_name):
    feature_df = pd.DataFrame(claims_df.MemberID)
    feature_df[feature_name] = 0
    
    i = 0
    n = len(feature_df)
    # count and sum
    for mem_id in feature_df.MemberID:
        feature_df.loc[feature_df.MemberID == mem_id, feature_name] = \
            df[df.MemberID == mem_id][feature_name].sum()
        
        # print out
        i += 1
        if i % 1000 == 0:
            print(i, '/', n)
    
    print('DONE')
    return feature_df

## 1. Import

In [59]:
DrugCount_df = pd.read_csv('data/hhp_release3/DrugCount.csv').query("Year == 'Y1' or Year == 'Y2'")
LabCount_df = pd.read_csv('data/hhp_release3/LabCount.csv').query("Year == 'Y1' or Year == 'Y2'")

# Replace DrugCount 7+ with 7
DrugCount_df.DrugCount.replace({'7+': '7'}, inplace=True)

# Replace LabCount 10+ with 10
LabCount_df.LabCount.replace({'10+': '10'}, inplace=True)

# Downcast to minimum int data type
DrugCount_df.DrugCount = pd.to_numeric(DrugCount_df.DrugCount, downcast='integer')
LabCount_df.LabCount = pd.to_numeric(LabCount_df.LabCount, downcast='integer')

## 2. Count sum

### 2.1. DrugCount

In [54]:
DrugCountY1_df = Count_summing(DrugCount_df.query("Year == 'Y1'"), Claims_countY1_df,'DrugCount')
DrugCountY2_df = Count_summing(DrugCount_df.query("Year == 'Y2'"), Claims_countY2_df,'DrugCount')

# Save to CSV
DrugCountY1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/DrugCount_Y1.csv', index=False)
DrugCountY2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/DrugCount_Y2.csv', index=False)

1000 / 76038
2000 / 76038
3000 / 76038
4000 / 76038
5000 / 76038
6000 / 76038
7000 / 76038
8000 / 76038
9000 / 76038


KeyboardInterrupt: 

### 2.2. LabCount

In [61]:
LabCountY1_df = Count_summing(LabCount_df.query("Year == 'Y1'"), Claims_countY1_df,'LabCount')
LabCountY2_df = Count_summing(LabCount_df.query("Year == 'Y2'"), Claims_countY2_df,'LabCount')

# Save to CSV
LabCountY1_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/LabCount_Y1.csv', index=False)
LabCountY2_df.sort_values('MemberID').to_csv(path_or_buf='processed_data/LabCount_Y2.csv', index=False)

1000 / 76038
2000 / 76038
3000 / 76038
4000 / 76038
5000 / 76038
6000 / 76038
7000 / 76038
8000 / 76038
9000 / 76038
10000 / 76038
11000 / 76038
12000 / 76038
13000 / 76038
14000 / 76038
15000 / 76038
16000 / 76038
17000 / 76038
18000 / 76038
19000 / 76038
20000 / 76038
21000 / 76038
22000 / 76038
23000 / 76038
24000 / 76038
25000 / 76038
26000 / 76038
27000 / 76038
28000 / 76038
29000 / 76038
30000 / 76038
31000 / 76038
32000 / 76038
33000 / 76038
34000 / 76038
35000 / 76038
36000 / 76038
37000 / 76038
38000 / 76038
39000 / 76038
40000 / 76038
41000 / 76038
42000 / 76038
43000 / 76038
44000 / 76038
45000 / 76038
46000 / 76038
47000 / 76038
48000 / 76038
49000 / 76038
50000 / 76038
51000 / 76038
52000 / 76038
53000 / 76038
54000 / 76038
55000 / 76038
56000 / 76038
57000 / 76038
58000 / 76038
59000 / 76038
60000 / 76038
61000 / 76038
62000 / 76038
63000 / 76038
64000 / 76038
65000 / 76038
66000 / 76038
67000 / 76038
68000 / 76038
69000 / 76038
70000 / 76038
71000 / 76038
72000 / 76038
7

# 5 & 6. Label

Drop ClaimsTruncated, sort based on MemberID and save

In [3]:
# Save to CSV
pd.read_csv('data/HHP_release3/DaysInHospital_Y2.csv').sort_values('MemberID').to_csv(path_or_buf='processed_data/DaysInHos_Y2.csv', index=False)
pd.read_csv('data/HHP_release3/DaysInHospital_Y3.csv').sort_values('MemberID').to_csv(path_or_buf='processed_data/DaysInHos_Y3.csv', index=False)